Cross language inference in R with the Python and Julia ONNX model

In [ ]:
Sys.setenv(RETICULATE_PYTHON = "/usr/bin/python3")
# Load library AFTER setting path
library(reticulate)

# Show Python config
py_config()

In [ ]:
onnxruntime <- import("onnxruntime")
print(onnxruntime$`__version__`)
np <- import("numpy")
pd <- import("pandas")

Python ONNNX Model 

In [ ]:
library(reticulate)

main <- function() {
  model_path <- "/tempcnn_py_model.onnx"
  x_path     <- "/X_frh04.npy"
  y_path     <- "/y_true_frh04.npy"
  fid_path   <- "/field_ids_frh04.npy"
  map_path   <- "/classmapping_final_cleaned.csv"
  max_rows   <- 10
  
  
  Sys.setenv(OMP_NUM_THREADS = "1", MKL_NUM_THREADS = "1")

  ort <- import("onnxruntime")
  np  <- import("numpy", convert = FALSE)
  pd  <- import("pandas", convert = TRUE)

  for (path in c(model_path, x_path, y_path, fid_path, map_path)) {
    if (!file.exists(path)) stop(paste("File not found:", path))
  }

  cat("Loading ONNX model with explicit thread counts...\n")
  so <- ort$SessionOptions()
  so$intra_op_num_threads <- as.integer(Sys.getenv("ORT_INTRA_OP_THREADS", "1"))
  so$inter_op_num_threads <- as.integer(Sys.getenv("ORT_INTER_OP_THREADS", "1"))
  session <- ort$InferenceSession(model_path, so)

  cat(" Loading input data...\n")
  X <- np$load(x_path, allow_pickle = TRUE)
  y_true <- np$load(y_path, allow_pickle = TRUE)
  field_ids <- np$load(fid_path, allow_pickle = TRUE)
  X_shape <- py_to_r(X$shape)

if (py_to_r(X$dtype$name) != "float32") {
  cat("Converting input data from", py_to_r(X$dtype$name), "to float32\n")
  X <- np$array(X, dtype = "float32")
}
  # Load class mapping
  classmap <- pd$read_csv(map_path)
  
  py$df <- classmap


py_run_string("
df_unique = df.drop_duplicates(subset='id')
df_sorted = df_unique.sort_values('id')
classnames = df_sorted['classname'].tolist()
")
classnames <- py$classnames
  input_name <- session$get_inputs()[[1]]$name
  cat(" Running inference on", as.integer(X_shape[[1]]), "samples...\n")

  result <- session$run(NULL, dict(input = X))
  logits <- result[[1]]
  logits_r <- logits
  softmax <- function(x) {
    e_x <- exp(x - apply(x, 1, max))
    e_x / rowSums(e_x)
  }

  probs <- softmax(logits_r)
  y_pred <- apply(probs, 1, which.max) - 1

  
  y_true <- py_to_r(y_true)
  field_ids <- py_to_r(field_ids)

  
  cat("\n Inference complete! Showing", max_rows, "results:\n\n")
  for (i in seq_len(min(max_rows, length(y_pred)))) {
    cat("Field ID:", field_ids[i], "\n")
    cat("   True:", y_true[i], "→", classnames[[y_true[i] + 1]], "\n")
    cat("   Pred:", y_pred[i], "→", classnames[[y_pred[i] + 1]], "\n")
    cat("   Probs:", round(probs[i, ], 4), "\n")
    cat("------------------------------------------------------------\n")
  }

  
  accuracy <- mean(y_true == y_pred) * 100
  cat(sprintf("\n Overall Accuracy: %.2f%%\n", accuracy))
}


# Run it
main()


Julia ONNX Model

In [ ]:
main <- function() {
  model_path <- "/tempcnn_julia_model.onnx"
  x_path     <- "/X_frh04.npy"
  y_path     <- "/y_true_frh04.npy"
  fid_path   <- "/field_ids_frh04.npy"
  map_path   <- "/classmapping_final_cleaned.csv"
  max_rows   <- 10
  
  Sys.setenv(OMP_NUM_THREADS = "1", MKL_NUM_THREADS = "1")

  ort <- import("onnxruntime")
  np  <- import("numpy", convert = FALSE)
  pd  <- import("pandas", convert = TRUE)

  for (path in c(model_path, x_path, y_path, fid_path, map_path)) {
    if (!file.exists(path)) stop(paste("File not found:", path))
  }

  cat("Loading ONNX model with explicit thread counts...\n")
  so <- ort$SessionOptions()
  so$intra_op_num_threads <- as.integer(Sys.getenv("ORT_INTRA_OP_THREADS", "1"))
  so$inter_op_num_threads <- as.integer(Sys.getenv("ORT_INTER_OP_THREADS", "1"))
  session <- ort$InferenceSession(model_path, so)


  cat(" Loading input data...\n")
  X <- np$load(x_path, allow_pickle = TRUE)
  y_true <- np$load(y_path, allow_pickle = TRUE)
  field_ids <- np$load(fid_path, allow_pickle = TRUE)
  X_shape <- py_to_r(X$shape)
  
if (py_to_r(X$dtype$name) != "float32") {
  cat("Converting input data from", py_to_r(X$dtype$name), "to float32\n")
  X <- np$array(X, dtype = "float32")
  
}

    X <- np$transpose(X, c(0L, 2L, 1L))$astype("float32")

  # Load class mapping
  classmap <- pd$read_csv(map_path)
  py$df <- classmap

py_run_string("
df_unique = df.drop_duplicates(subset='id')
df_sorted = df_unique.sort_values('id')
classnames = df_sorted['classname'].tolist()
")

# list back to R
classnames <- py$classnames

 input_name <- session$get_inputs()[[1]]$name
cat("Input name from ONNX model:", input_name, "\n")
cat(" Running inference on", as.integer(X_shape[[1]]), "samples...\n")

feed <- dict()
feed[[input_name]] <- X   
result <- session$run(NULL, feed)


  logits <- result[[1]]
  logits_r <- logits  
  softmax <- function(x) {
    e_x <- exp(x - apply(x, 1, max))
    e_x / rowSums(e_x)
  }

  probs <- softmax(logits_r)
  y_pred <- apply(probs, 1, which.max) - 1

  
  y_true <- py_to_r(y_true)
  field_ids <- py_to_r(field_ids)

  
  cat("\n Inference complete! Showing", max_rows, "results:\n\n")
  for (i in seq_len(min(max_rows, length(y_pred)))) {
    cat("Field ID:", field_ids[i], "\n")
    cat("   True:", y_true[i], "→", classnames[[y_true[i] + 1]], "\n")
    cat("   Pred:", y_pred[i], "→", classnames[[y_pred[i] + 1]], "\n")
    cat("   Probs:", round(probs[i, ], 4), "\n")
    cat("------------------------------------------------------------\n")
  }

  
  accuracy <- mean(y_true == y_pred) * 100
  cat(sprintf("\n Overall Accuracy: %.2f%%\n", accuracy))
}


# Run it
main()
